# Compute diffusion scores in R

### Imports

In [1]:
library(igraph)
library(MASS)


Attaching package: ‘igraph’

The following objects are masked from ‘package:stats’:

    decompose, spectrum

The following object is masked from ‘package:base’:

    union



### Import graph

In [2]:
g = read_graph('/Users/jmarinllao/Documents/CREB/diffupy/notebooks/04_unit_testing/_graph.gml', format = c("gml"))

### Helpers

In [3]:
#' In which format is the input?
#'
#' Tell apart vector, matrix or list of matrices
#'
#' @param x object to evaluate
#'
#' @return character: vector, matrix or list.
#'
#' @examples
#' data(graph_toy)
#' diffuStats:::which_format(graph_toy$input_vec)
#' diffuStats:::which_format(graph_toy$input_mat)
which_format <- function(x) {
    if (is.numeric(x) & is.null(dim(x))) return("vector")
    if (is.numeric(x)) return("matrix")
    if (is.list(x)) return("list")

    stop("Non-recognised input scores format, object of class: ", class(x))
}

In [4]:
#' Convert input to list format
#'
#' Convert any input to list format
#'
#' @param scores object to reformat
#' @param dummy_column,dummy_list character, names for the dummy columns/items
#'
#' @return scores in list format
#'
#' @examples
#' data(graph_toy)
#' x_v <- diffuStats:::to_list(graph_toy$input_vec)
#' x_m <- diffuStats:::to_list(graph_toy$input_mat)
to_list <- function(scores, dummy_column = "X1", dummy_list = "X1") {
    s_format <- which_format(scores)

    if (s_format == "vector") {
        names_scores <- names(scores)
        scores <- matrix(scores, ncol = 1)
        rownames(scores) <- names_scores
        colnames(scores) <- dummy_column
    }
    if (s_format %in% c("matrix", "vector")) {
        scores <- list(scores)
        names(scores) <- dummy_list
    }
    return(scores)

    stop(
        "Non-recognised input scores format, object of class: ",
        class(s_format))
}

In [5]:
#' Convert list format to desired format
#'
#' Convert any list format to the convenient one
#'
#' @param scores list to reformat
#' @param x character, desired format
#'
#' @return scores in desired format
#'
#' @examples
#' data(graph_toy)
#' x_v <- diffuStats:::to_x_from_list(
#'     diffuStats:::to_list(graph_toy$input_vec), "vector")
#' x_m <- diffuStats:::to_x_from_list(
#'     diffuStats:::to_list(graph_toy$input_vec), "matrix")
to_x_from_list <- function(scores, x) {
    if (x == "list") return(scores)
    if (x == "matrix") return(scores[[1]])
    if (x == "vector")
        return(setNames(scores[[1]][, 1], rownames(scores[[1]])))
}

### Checkers

In [6]:
#' Sanity checks for input
#'
#' .check_scores ensures that scores are suitable for diffusion
#'
#' @param scores scores to check
#'
#' @return Functions return \code{invisible()} but throw
#' warnings and errors as side effect
#' @rdname checks
#'
#' @importFrom stats sd
#' @examples
#' data(graph_toy)
#' diffuStats:::.check_scores(diffuStats:::to_list(graph_toy$input_mat))
.check_scores <- function(scores) {
    form <- which_format(scores)
    scores_names <- names(scores)

    if (is.null(scores_names))
        stop(
            "Scores must be a named list, ",
            " but supplied list contains no names."
        )

    plyr::l_ply(
        scores_names,
        function(mat_name) {
            mat <- scores[[mat_name]]

            if (!is.numeric(mat) & !("dgCMatrix" %in% class(mat))) {
                stop(
                    "The scores in background ",
                    mat_name,
                    " are not numeric!"
                )
            }
            if (any(is.na(mat))) {
                stop(
                    "Scores input cannot contain NA! ",
                    "But background ",
                    mat_name,
                    " does!")
            }
            if (is.null(rownames(mat)))
                stop(
                    "The scores in background ",
                    mat_name,
                    " must have rownames ",
                    "according to the scored nodes!"
                )
            if (is.null(colnames(mat)))
                stop(
                    "The scores in background ",
                    mat_name,
                    " must have colnames ",
                    "to differentiate score sets!"
                )

            std <- apply(mat, 2, stats::sd)
            std_zero <- which(std == 0)
            std_na <- which(is.na(std))
            

            if (length(std_na))
                warning(
                    "Standard deviation in background ",
                    mat_name,
                    " is NA in columns: ",
                    paste(std_na, collapse = ",")
                )
            if (length(std_zero))
                warning(
                    "Standard deviation in background ",
                    mat_name,
                    " is 0 in columns: ",
                    paste(std_zero, collapse = ",")
                )
        }
    )
    invisible()
}

#' Available methods for diffusion
#'
#' .available_methods is a character vector with the implemented scores
#'
#' @rdname checks
.available_methods <- c("raw", "ml", "gm", "mc", "z", "ber_s", "ber_p")

#' Check method sanity
#'
#' .check_method ensures that 'method' is a valid character
#'
#' @param method object to test
#'
#' @rdname checks
#'
#' @examples
#' diffuStats:::.check_method("raw")
.check_method <- function(method) {
    if (!is.character(method) & !is.factor(method))
        stop(
            "The supplied 'method' must be a character, ",
            "but the one supplied is a ",
            class(method))

    if (length(method) > 1)
        stop(
            "Only one 'method' can be supplied at once, ",
            "but you supplied ",
            length(method))


    if (!(method %in% .available_methods))
        stop(
            "The available methods are ",
            paste(.available_methods, collapse = ","),
            " but you supplied '",
            method,
            "', which is not implemented")

    invisible()
}

#' Check metric sanity
#'
#' .check_metric ensures that 'metric' is a valid list of metric functions
#'
#' @param metric object to test
#'
#' @rdname checks
#'
#' @examples
#' diffuStats:::.check_metric(list(auc = metric_fun(curve = "ROC")))
.check_metric <- function(metric) {
    if (!is.list(metric))
        stop(
            "'metric' must be a list of metrics, ",
            "but it is not a list")
    names_metric <- names(metric)
    if (is.null(names_metric))
        stop(
            "'metric' must be a named list, but supplied list ",
            "has no names.")

    plyr::l_ply(
        names_metric,
        function(met) {
            fun_met <- (metric[[met]])
            if (!is.function(fun_met))
                stop(
                    "In the 'metric' list supplied, the metric named",
                    met,
                    " is not actually a function, but a ",
                    paste(class(fun_met), collapse = ","))
            fun_formals <- formals(fun_met)
            if (length(fun_formals) < 2)
                stop(
                    "In the 'metric' list supplied, the metric named",
                    met,
                    " has less than 2 arguments: ",
                    paste(names(fun_formals), collapse = ",")
                )
            if (length(fun_formals) > 2)
                warning(
                    "In the 'metric' list supplied, the metric named",
                    met,
                    " has more than 2 arguments: ",
                    paste(names(fun_formals), collapse = ","),
                    ". Make sure this is a proper metric function."
                )
            dummy_call <- do.call(
                fun_met,
                list(c(1, 0, 0), c(.75, .25, .2)))
            if (length(dummy_call) > 1)
                warning(
                    "A sanity check call of the 'metric' function ",
                    met,
                    " returns a vector with length greater than 1. ",
                    "Is the metric correct?"
                )
        }
    )
    invisible()
}

#' Check graph sanity
#'
#' .check_graph ensures that 'graph' is a valid \code{igraph} object
#'
#' @param graph object to test
#'
#' @rdname checks
#'
#' @examples
#' data(graph_toy)
#' diffuStats:::.check_graph(graph_toy)
.check_graph <- function(graph) {
    if (missing(graph) | is.null(graph)) return(invisible())
    if (!is.igraph(graph))
        stop("'graph' must be an igraph object")

    node_name <- V(graph)$name
    if (is.null(node_name))
        stop(
            "'graph' must have node names! ",
            "Set them through V(graph)$name <- ")
    
        
    if (any(is.na(node_name)))
        stop(
            "'graph' cannot have NA as node names")
    if (any(duplicated(V(graph)$name)))
        stop(
            "'graph' has non-unique names! ",
            "Please check that the names are unique."
        )
    if (is.directed(graph))
        warning(
            "'graph' should be an undirected igraph object. ",
            "You should use as.undirected"
        )
    edge_weight <- E(graph)$weight # can be null
    if (!is.null(edge_weight)) {
        if (any(is.na(edge_weight)))
            stop("'graph' cannot contain NA edge weights")
        if (any(edge_weight < 0))
            warning("'graph' should not contain negative edge weights")
    }

    invisible()
}


#' Check kernel sanity
#'
#' .check_K ensures that 'K' is a formally valid kernel.
#' Does not check for spd
#'
#' @param K object to test
#'
#' @rdname checks
#'
#' @examples
#' data(graph_toy)
#' diffuStats:::.check_K(regularisedLaplacianKernel(graph_toy))
.check_K <- function(K) {
    if (!is.matrix(K))
        stop("'K' must be a matrix")

    if (!is.numeric(K))
        stop("'K' must be a numeric matrix, but it is not numeric.")
    if (any(is.na(K)))
        stop("'K' cannot have NA as values")

    if (nrow(K) != ncol(K))
        stop(
            "'K' must be a square matrix, but it has ",
            nrow(K),
            " rows and ",
            ncol(K),
            " columns.")

    rown <- rownames(K)
    coln <- colnames(K)

    if (is.null(rown))
        stop("'K' kernel must have rownames!")
    if (is.null(coln))
        stop("'K' kernel must have colnames!")
    if (any(is.na(c(rown, coln))))
        stop("'K' dimnames cannot be NA")
    if (any(rown != coln))
        stop("'K' rownames and colnames must coincide")
    if (any(duplicated(rown)))
        stop("'K' cannot contain duplicated rownames/colnames")

    invisible()
}

### Kernels

In [7]:
regularisedLaplacianKernel <- function(
    graph,
    sigma2 = 1,
    add_diag = 1,
    normalized = FALSE) {
    if (is.directed(graph)) stop("'graph' must be undirected")
    
    L <- graph.laplacian(graph = graph, normalized = normalized)

    RL <- sigma2*L
    Matrix::diag(RL) <- Matrix::diag(RL) + add_diag
    
    ans <- as.matrix(Matrix::solve(RL, sparse = FALSE))
    rownames(ans) <- colnames(ans) <- V(graph)$name
    ans
}

### Diffuse scores

### Diffuse raw 

In [8]:
#' Diffuse scores on a network
#'
#' Function \code{diffuse} takes a network in
#' \pkg{igraph} format and an initial state
#' to score all the nodes in the network.
#'
#' @param graph \pkg{igraph} object for the diffusion
#' @param scores list of score matrices. For a single input with a
#' single background, supply a list with a vector column
#' @param z logical, should z-scores be computed instead of raw scores?
#' @param K optional matrix, precomputed diffusion kernel
#' @param ... currently ignored arguments
#'
#' @return A list of scores, with the same length and
#' dimensions as \code{scores}
#'
#' @examples
#' # Using a list as input (needed)
#' data(graph_toy)
#' list_input <- list(myInput1 = graph_toy$input_mat)
#' diff_raw <- diffuse_raw(
#'     graph = graph_toy,
#'     scores = list_input)
#' diff_z <- diffuse_raw(
#'     graph = graph_toy,
#'     scores = list_input,
#'     z = TRUE)
#' @import igraph
#' @export
diffuse_raw <- function(
    graph,
    scores,
    z = FALSE,
    K = NULL,
    ...) {
    # sanity checks
    .check_scores(scores)

    # Kernel matrix
    if (is.null(K)) {
        .check_graph(graph)
        message(
            "Kernel not supplied. ",
            "Computing regularised Laplacian kernel ...")
        K <- regularisedLaplacianKernel(graph = graph)
        message("Done")
    } else {
        .check_K(K)
        message("Using supplied kernel matrix...")
    }

    # Compute scores
    ans.all <- plyr::llply(
        stats::setNames(names(scores), names(scores)),
        function(scores.name) {
            # match indices (NO NAMES, careful)
            bkgd.names <- rownames(scores[[scores.name]])
            input.names <- colnames(scores[[scores.name]])

            # input scores
            scores.mat <- methods::as(scores[[scores.name]], "sparseMatrix")

            # raw scores
            diff.raw <- K[, bkgd.names] %*% scores.mat

            rownames(diff.raw) <- rownames(K)
            colnames(diff.raw) <- input.names

            # Return base matrix if it is raw
            # Continue if we want z-scores
            if (z == FALSE) return(as.matrix(diff.raw))

            # If we want z-scores, must compute rowmeans and rowmeans2
            .rowSums <- Matrix::rowSums(K[, bkgd.names])
            .rowSums2 <- Matrix::rowSums(K[, bkgd.names] ** 2)


            # Constant terms over columns
            n <- length(bkgd.names)
            const_mean <- .rowSums/n
            const_var <-  (n*.rowSums2 - .rowSums**2)/((n - 1)*(n**2))
            
            
            diff.z <- sapply(
                1:ncol(diff.raw),
                function(col_ind) {
                    col_in <- scores.mat[, col_ind]
                    col_raw <- diff.raw[, col_ind]
                    
                    s1 <- sum(col_in)
                    s2 <- sum(col_in**2)

                    # means and vars depend on first and second moments
                    # of the input. This should be valid for non-binary
                    # inputs as well
                    score_means <- const_mean*s1
                    score_vars <- const_var*(n*s2 - s1**2)
                    
                    (col_raw - score_means)/sqrt(score_vars)
                }
            )
            # Give correct names
            dimnames(diff.z) <- dimnames(diff.raw)

            # This is already a base matrix
            return(diff.z)
        }
    )

    return(ans.all)
}

### Diffuse

In [9]:
#' Diffuse scores on a network
#'
#' Function \code{diffuse} takes a network in
#' \pkg{igraph} format and an initial state
#' to score all the nodes in the network.
#'
#' @param graph \pkg{igraph} object for the diffusion
#' @param scores list of score matrices. For a single input with a
#' single background, supply a list with a vector column
#' @param z logical, should z-scores be computed instead of raw scores?
#' @param K optional matrix, precomputed diffusion kernel
#' @param ... currently ignored arguments
#'
#' @return A list of scores, with the same length and
#' dimensions as \code{scores}
#'
#' @examples
#' # Using a list as input (needed)
#' data(graph_toy)
#' list_input <- list(myInput1 = graph_toy$input_mat)
#' diff_raw <- diffuse_raw(
#'     graph = graph_toy,
#'     scores = list_input)
#' diff_z <- diffuse_raw(
#'     graph = graph_toy,
#'     scores = list_input,
#'     z = TRUE)
#' @import igraph
#' @export
diffuse <- function(
    graph,
    scores,
    method,
    ...) {
    # check sanity
    .check_method(method)

    # For data reshaping
    format_scores <- which_format(scores)
    scores <- to_list(scores)

    # Check if we have a graph or a kernel
    if (!missing("graph")) {
        format_network <- "graph"
    } else {
        if (!("K" %in% names(list(...))))
            stop("Neither a graph 'graph' or a kernel 'K' were provided")
        format_network <- "kernel"
    }

    if (method == "raw") {
        ans <- (diffuse_raw(graph = graph, scores = scores, ...))
    }
    if (method == "ml") {
        scores_ml <- lapply(
            scores,
            function(mat) {
                if (!all(as.numeric(mat) %in% c(0, 1))) 
                    stop("Input scores for ", method, " must be binary")
                mat[mat == 0] <- -1
                mat
            }
        )
        ans <- (diffuse_raw(graph = graph, scores = scores_ml, ...))
    }
    if (method == "gm") {
        scores_gm <- lapply(
            scores,
            function(mat) {
                if (!all(as.numeric(mat) %in% c(0, 1))) 
                    stop("Input scores for ", method, " must be binary")
                # Have to match rownames with background
                # If the kernel is provided...
                if (format_network == "graph") {
                    names_ordered <- V(graph)$name
                } else if (format_network == "kernel") {
                    names_ordered <- rownames(list(...)[["K"]])
                }
                
                # If the graph is defined...
                ids_nobkgd <- setdiff(names_ordered, rownames(mat))
                n_tot <- length(names_ordered)
                n_bkgd <- nrow(mat)
                n_col <- ncol(mat)

                # normalisation has to be performed
                # for each column, as it depends
                # on the number of positives and negatives...
                # n_pos and n_neg are vectors counting the number of 
                # positives and negatives in each column
                n_pos <- colSums(mat)
                n_neg <- n_bkgd - n_pos
                # biases
                p <- (n_pos - n_neg)/n_tot
                
                mat[mat == 0] <- -1
                # add biases (each column has its bias)
                mat.rbind <- matrix(
                    nrow = n_tot - n_bkgd, 
                    ncol = n_col, 
                    data = rep(p, each = n_tot - n_bkgd))
                rownames(mat.rbind) <- ids_nobkgd
                
                mat <- rbind(as.matrix(mat), mat.rbind)
                
                # sort the names as in the original graph
                mat[names_ordered, , drop = FALSE]
            }
        )
        ans <- (diffuse_raw(graph = graph, scores = scores_gm, ...))
    }

    # Monte-Carlo simulations
    if (method == "mc") {
        ans <- (diffuse_mc(graph = graph, scores = scores, ...))
    }

    # z scores
    if (method == "z") {
        ans <- (diffuse_raw(graph = graph, scores = scores, z = TRUE, ...))
    }

    # Bersanelli's scores (s)
    if (method == "ber_s") {
        list_dots <- list(...)

        if ("eps" %in% names(list_dots)) {
            eps <- list_dots$eps
        } else {
            eps <- 1
        }
        # Compute final state
        scores_raw <- diffuse_raw(graph = graph, scores = scores, ...)

        scores_ber_s <- plyr::llply(
            setNames(names(scores), names(scores)),
            function(scores_name) {
                # each list can have a different background...
                # nodes outside the background will be
                # assigned a prior score of 0
                mat_out <- scores_raw[[scores_name]]
                mat_in <- scores[[scores_name]]

                # matrix with correct dimnames but populated with eps
                mat_in_fill <- Matrix::Matrix(
                    data = eps,
                    nrow = nrow(mat_out),
                    ncol = ncol(mat_out),
                    dimnames = dimnames(mat_out))

                # add the original input: only in the rows that match
                mat_in_fill[rownames(mat_in), ] <-
                    mat_in_fill[rownames(mat_in), ] + mat_in

                as.matrix(mat_out/mat_in_fill)
            }
        )
        ans <- (scores_ber_s)
    }

    # Bersanelli's scores (p)
    if (method == "ber_p") {
        # Compute final state
        scores_raw <- diffuse_raw(graph = graph, scores = scores, ...)

        # Compute p
        scores_mc <- diffuse_mc(
            graph = graph,
            scores = scores,
            oneminusHeatRank = FALSE,
            ...)

        scores_ber_p <- plyr::llply(
            setNames(names(scores), names(scores)),
            function(scores_name) {
                s_mc <- scores_mc[[scores_name]]
                s_raw <- scores_raw[[scores_name]]

                -log10(s_mc)*s_raw
            }
        )
        ans <- (scores_ber_p)
    }
    if (!exists("ans")) {
        message(
            "The specified method ",
            method,
            " is not implemented and will return NULL")
        return(invisible())
    }

    message("All done")
    # reshape back to original and return
    return(to_x_from_list(ans, format_scores))
}


### Generate scores example

In [10]:
input_mat <- matrix(rep(0, times=100), 100, 1)
input_mat <- cbind(input_mat, sample(0:1, 100, replace=TRUE))

names <-vertex_attr(g)$name
rownames(input_mat) <- names
colnames(input_mat) <- c('input1', 'input2')

input_mat['V3', 'input1'] <- 1
input_mat['V20', 'input1'] <- 1
input_mat['V30', 'input1'] <- 1

input_mat

write.csv(input_mat, file = "scores_test/input_methods_scores.csv")

,input1,input2
V1,0,0
V2,0,0
V3,1,1
V4,0,1
V5,0,0
V6,0,1
V7,0,1
V8,0,1
V9,0,1
V10,0,1


## Diffuse scores

### raw

In [11]:
b <- list(input1 = input_mat)
ans = diffuse(g, b, method = 'raw')
ans
write.csv(ans, file = "scores_test/output_raw_scores.csv")

Kernel not supplied. Computing regularised Laplacian kernel ...
Done
All done


,input1,input2
V1,0.04214556,0.4056519
V2,0.02803012,0.3632810
V3,0.07392846,0.4656894
V4,0.02882746,0.4894857
V5,0.02498394,0.3928873
V6,0.02694918,0.4541143
V7,0.04164558,0.4598546
V8,0.02889637,0.4470017
V9,0.03051561,0.4796102
V10,0.02342762,0.5328166


### z-scores

In [12]:
b <- list(input1 = input_mat)
ans = diffuse(g, b, method = 'z')
ans
write.csv(ans, file = "scores_test/output_z_scores.csv")

Kernel not supplied. Computing regularised Laplacian kernel ...
Done
All done


,input1,input2
V1,0.57290050,-0.39573183
V2,-0.11334607,-1.32279086
V3,4.08043369,1.14228413
V4,-0.09238872,1.61502235
V5,-0.40922098,-1.04326021
V6,-0.25141027,0.68472554
V7,1.11370470,0.98377375
V8,-0.05888038,0.31254747
V9,0.03712219,1.23071287
V10,-0.25779560,1.38960436


### ml

In [13]:
b <- list(input1 = input_mat)
ans = diffuse(g, b, method = 'ml')
ans
write.csv(ans, file = "scores_test/output_ml_scores.csv")

Kernel not supplied. Computing regularised Laplacian kernel ...
Done
All done


,input1,input2
V1,-0.9157089,-0.18869615
V2,-0.9439398,-0.27343794
V3,-0.8521431,-0.06862124
V4,-0.9423451,-0.02102854
V5,-0.9500321,-0.21422545
V6,-0.9461016,-0.09177134
V7,-0.9167088,-0.08029078
V8,-0.9422073,-0.10599661
V9,-0.9389688,-0.04077966
V10,-0.9531448,0.06563321


### gm

In [14]:
b <- list(input1 = input_mat)
ans = diffuse(g, b, method = 'gm')
ans
write.csv(ans, file = "scores_test/output_gm_scores.csv")

Kernel not supplied. Computing regularised Laplacian kernel ...
Done
All done


,input1,input2
V1,-0.9157089,-0.18869615
V2,-0.9439398,-0.27343794
V3,-0.8521431,-0.06862124
V4,-0.9423451,-0.02102854
V5,-0.9500321,-0.21422545
V6,-0.9461016,-0.09177134
V7,-0.9167088,-0.08029078
V8,-0.9422073,-0.10599661
V9,-0.9389688,-0.04077966
V10,-0.9531448,0.06563321
